In [1]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [2]:
!pip install cartopy

In [3]:
!pip install gTTS

In [4]:
!pip install sounddevice

In [5]:
!pip install SpeechRecognition sounddevice
!apt-get install python3-pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-pyaudio is already the newest version (0.2.11-1.3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 50 not upgraded.


In [6]:
import time
from IPython.display import clear_output, display
import ipywidgets as widgets
import speech_recognition as sr
import threading
import re
from google.colab import output
import numpy as np
import sounddevice as sd

In [7]:
class ManuelTimer:
    def __init__(self):
        self.running = False
        self.start_time = 0
        self.recognizer = sr.Recognizer()
        self.is_listening = False

    def listen_for_commands(self):
        """Continuously listen for voice commands"""
        self.is_listening = True
        print("Ready when you ready! VIVAA:")
        print("- 'start timer X minutes' or 'start timer X seconds'")
        print("- 'start stopwatch'")
        print("- 'stop'")

        while self.is_listening:
            try:
                with sr.Microphone() as source:
                    self.recognizer.adjust_for_ambient_noise(source, duration=0.5)
                    audio = self.recognizer.listen(source)
                    text = self.recognizer.recognize_google(audio).lower()
                    print(f"Heard: {text}")

                    # Parse commands
                    if "start timer" in text:
                        # Extract time values
                        minutes = 0
                        seconds = 0
                        time_values = re.findall(r'(\d+)\s*(minute|second)', text)
                        for value, unit in time_values:
                            if 'minute' in unit:
                                minutes = int(value)
                            elif 'second' in unit:
                                seconds = int(value)
                        if minutes > 0 or seconds > 0:
                            threading.Thread(target=self.countdown, args=(minutes, seconds)).start()

                    elif "start stopwatch" in text:
                        threading.Thread(target=self.stopwatch).start()

                    elif "stop" in text:
                        self.running = False
                        print("Timer stopped!")

            except sr.UnknownValueError:
                pass
            except sr.RequestError:
                print("Sorry, there was an error with the speech recognition service")

    def countdown(self, minutes, seconds):
        """Countdown timer function"""
        self.running = True
        total_seconds = minutes * 60 + seconds

        while total_seconds > 0 and self.running:
            clear_output(wait=True)
            m, s = divmod(total_seconds, 60)
            print("Voice commands active! Say 'stop' to end timer")
            print(f'Time remaining: {int(m):02d}:{int(s):02d}')
            time.sleep(1)
            total_seconds -= 1

        if total_seconds == 0:
            print('\nTime is up! 🔔')
            # Generate beep sound
            sample_rate = 44100
            duration = 1  # seconds
            frequency = 440  # Hz
            t = np.linspace(0, duration, int(sample_rate * duration))
            samples = 0.5 * np.sin(2 * np.pi * frequency * t)
            sd.play(samples, sample_rate)

    def stopwatch(self):
        """Stopwatch function"""
        self.running = True
        start_time = time.time()

        while self.running:
            clear_output(wait=True)
            elapsed_time = time.time() - start_time
            m, s = divmod(int(elapsed_time), 60)
            h, m = divmod(m, 60)
            print("Voice commands active! Say 'stop' to end stopwatch")
            print(f'Time elapsed: {h:02d}:{m:02d}:{s:02d}')
            time.sleep(0.1)

In [8]:
# First, install all required dependencies
!apt-get update && apt-get install -y portaudio19-dev python3-pyaudio
!pip install SpeechRecognition pyaudio sounddevice

import time
from IPython.display import clear_output, display
import ipywidgets as widgets
import speech_recognition as sr
import threading
import re
import numpy as np
import sounddevice as sd
from google.colab import output

# Check audio devices
!python -m sounddevice

class VoiceTimer:
    def __init__(self):
        self.running = False
        self.start_time = 0
        self.recognizer = sr.Recognizer()
        self.is_listening = False

    def listen_for_commands(self):
        """Continuously listen for voice commands"""
        self.is_listening = True

        # First, verify microphone access
        try:
            with sr.Microphone() as source:
                print("Microphone initialized successfully!")
                print("\nVoice activation ready! Try saying:")
                print("- 'start timer X minutes' or 'start timer X seconds'")
                print("- 'start stopwatch'")
                print("- 'stop'")

                while self.is_listening:
                    try:
                        audio = self.recognizer.listen(source, timeout=1, phrase_time_limit=5)
                        text = self.recognizer.recognize_google(audio).lower()
                        print(f"Heard: {text}")

                        if "start timer" in text:
                            minutes = 0
                            seconds = 0
                            time_values = re.findall(r'(\d+)\s*(minute|second)', text)
                            for value, unit in time_values:
                                if 'minute' in unit:
                                    minutes = int(value)
                                elif 'second' in unit:
                                    seconds = int(value)
                            if minutes > 0 or seconds > 0:
                                threading.Thread(target=self.countdown,
                                              args=(minutes, seconds)).start()

                        elif "start stopwatch" in text:
                            threading.Thread(target=self.stopwatch).start()

                        elif "stop" in text:
                            self.running = False
                            print("Timer stopped!")

                    except sr.WaitTimeoutError:
                        continue
                    except sr.UnknownValueError:
                        continue
                    except sr.RequestError as e:
                        print(f"Could not request results; {e}")

        except Exception as e:
            print(f"Error initializing microphone: {e}")
            print("\nTroubleshooting steps:")
            print("1. Make sure you've allowed microphone access in your browser")
            print("2. Check if your microphone is properly connected")
            print("3. Try restarting the runtime and running the code again")
            print("4. If using a virtual environment, make sure it has access to audio devices")

    def countdown(self, minutes, seconds):
        """Countdown timer function"""
        self.running = True
        total_seconds = minutes * 60 + seconds

        while total_seconds > 0 and self.running:
            clear_output(wait=True)
            m, s = divmod(total_seconds, 60)
            print("Voice commands active! Say 'stop' to end timer")
            print(f'Time remaining: {int(m):02d}:{int(s):02d}')
            time.sleep(1)
            total_seconds -= 1

        if total_seconds == 0:
            print('\nTime is up! 🔔')
            # Generate beep sound
            sample_rate = 44100
            duration = 1
            frequency = 440
            t = np.linspace(0, duration, int(sample_rate * duration))
            samples = 0.5 * np.sin(2 * np.pi * frequency * t)
            sd.play(samples, sample_rate)

    def stopwatch(self):
        """Stopwatch function"""
        self.running = True
        start_time = time.time()

        while self.running:
            clear_output(wait=True)
            elapsed_time = time.time() - start_time
            m, s = divmod(int(elapsed_time), 60)
            h, m = divmod(m, 60)
            print("Voice commands active! Say 'stop' to end stopwatch")
            print(f'Time elapsed: {h:02d}:{m:02d}:{s:02d}')
            time.sleep(0.1)

# Create and start the voice-activated timer
print("Initializing voice timer...")
timer = VoiceTimer()
voice_thread = threading.Thread(target=timer.listen_for_commands)
voice_thread.start()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading